Preparing the datasets from the Google Sheets to fit the format of the website

In [6]:
import numpy as np
import pandas as pd

In [7]:
# Ignore surplus columns and rows before the table, keep N/A symbols as they are
df = pd.read_csv("Dimensions_250512.csv", skiprows=2, keep_default_na=False).iloc[:, 1:-17]

# Kick out some of the columns as well as Out And Elicitation rows
df = df[df["Filter"].isin(["Elicitation", "Enabling", "In"])]
df.drop(columns=["Filter", "Motivations (Notes)", "Motivations"], inplace=True)
df.replace("", np.NaN, inplace=True)

# Move the "ID" column to the first position
df.rename(columns={"DB ID": "ID"}, inplace=True)
df = df[["ID"] + [col for col in df.columns if col != "ID"]]

# Custom title function with exceptions
def custom_title(text, exceptions, stop_words):
    words = text.split()
    titled_words = [word if word in exceptions else (word if word in stop_words else word.title()) for word in words]
    return ' '.join(titled_words)

# List of exceptions
exceptions = ["AR", "VR", "AR/VR", "MP3", "EMG", "EEG", "EOG", "mmWaveRadar", "ECG"]

# List of stop words
stop_words = ["and", "the", "of", "in", "on", "at", "with", "a", "an"]

# Apply sorting and custom title casing
for col in ['Location', 'Input Body Part', 'Gesture', 'Sensors', 'Resolution', 'Evaluation of Different Conditions', 'Evaluations of Different Settings', 'Earphone Type', 'Intended Applications', 'Keywords']:
    df[col] = df[col].apply(lambda x: ', '.join(sorted(custom_title(word, exceptions, stop_words) for word in x.split(", "))))
    
# Display the dataframe
df


,ID,Main Author,Year,Location,Input Body Part,Gesture,Sensors,No Additional Sensing,Number of Selected Gestures,Resolution,...,Evaluation of Different Conditions,Evaluations of Different Settings,Earphone Type,Development Stage,Real-Time Processing,On Device Processing,Intended Applications,Keywords,Abstract,Study Link
0,1,Weisenberger et al.,1987,Actuation,N/A,Vibration (Actuation),N/A,N/A,1,Fine,...,Sitting,Lab,Earbud,Research Prototype,N/A,N/A,"Accessibility, Health",N/A,A binaural earmold sound-to-tactile aid was co...,https://www.researchgate.net/publication/19575...
1,2,Brewster et al.,2003,Head Gestures and Pointing,Head,"Pitch, Roll","Accelerometer, Gyroscope, Magnetometer",Yes,1,Coarse,...,Sitting,Lab,Custom Device,Commercial,Yes,No,Device Input,"Gestural Interaction, Wearable Computing",Mobile and wearable computers present input/ou...,https://dl.acm.org/doi/abs/10.1145/642611.642694
2,3,Metzger et al.,2004,"Hand Gestures and Location, Head Gestures and ...","Hand, Head","Hold (Mid-Air), Roll, Slide (Mid-Air)","Accelerometer, Proximity Sensor",Yes,7,"Coarse, Semantic",...,Sitting,Lab,Headphone,Research Prototype,Yes,No,"Accessibility, Device Control, Music Player, P...",N/A,"We present FreeDigiter, an interface for mobil...",https://ieeexplore.ieee.org/document/1364684
3,4,Buil & Hollemans,2005,Ear and Earable,Hand,Press (Earable),Button,Yes,3,Semantic,...,Sitting,Lab,Earbud,Research Prototype,Yes,Yes,Music Player,N/A,The touch headphones are a solution for provid...,https://ieeexplore.ieee.org/abstract/document/...
4,5,Buil et al.,2005,Ear and Earable,"Hand, Wearable State","Attach Earbud, Hold (Earable), Remove Earbud, ...",Capacitive Sensor,Yes,5,Semantic,...,N/A,N/A,Earbud,Research Prototype,Yes,No,"Device Control, Music Player","Capacitive Touch Control, Headphones, MP3, Mus...",The Touch Headphones are meant for portable mu...,https://dl.acm.org/doi/abs/10.1145/1085777.108...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,114,Wang et al.,2024,Hand Gestures and Location,Hand,"Slide (Face), Tap (Face)","Accelerometer, Gyroscope",Yes,7,Semantic,...,Sitting,Lab,Earbud,Commercial,No,No,Authentification,"Adversarial Learning, Implicit Authentication,...",The surge in popularity of wireless headphones...,https://ieeexplore.ieee.org/abstract/document/...
119,115,Wang et al.,2024,Hand Gestures and Location,Hand,Write (Face),Microphone,Yes,36,Semantic,...,"Head Movement, Sitting, Standing, Walking","Car, Living Room, Office, Outdoors",Earbud,Research Prototype,No,No,"Device Input, Privacy","Acoustic Sensing, Earable, Face and Ear Intera...","As earables gain popularity, there emerges a n...",https://ieeexplore.ieee.org/document/10637602
120,116,Xie et al.,2024,Head Gestures and Pointing,Head,General Head Movement,"Accelerometer, Gyroscope",No,1,Semantic,...,"Cycling, Music, Noise, Sitting, Standing, Walk...","Lab, Outdoors",Earbud,Commercial,Yes,No,Authentification,"Earphones, Head Motion, Ubiquitous Computing, ...",With the growing reliance on digital systems i...,https://ieeexplore.ieee.org/document/10631065
121,117,Yang et al.,2024,"Ear and Earable, Hand Gestures and Location",Hand,"Approach (Mid-Air), Click (Mid-Air), Close (Mi...","Microphone, Speaker",Yes,10,Semantic,...,"Distance, Eating, Hydration, Music, Noise, Sit...",Lab,Earbud,Research Prototype,No,No,"AR/VR, Device Control, Health, Music Player","Acoustic Sensing, Gesture Detection, Wearable ...","We present MAF, a novel acoustic sensing appro...",https://dl.acm.org/doi/10.1145/3613904.3642437


In [8]:
# Define a dictionary with the current column names as keys and the new column names as values
new_column_names = {
    "ID": "ID",
    "Main Author": "First Author",
    "Year": "Year",
    "Location": " Location",
    "Input Body Part": "Input Body Part",
    "Gesture": "Gesture",
    "Sensors": "Sensing_PANEL_Sensors",
    "No Additional Sensing": "Sensing_PANEL_No Additional Sensing",
    "Number of Selected Gestures": "Interaction_PANEL_Number of Selected Gestures",
    "Resolution": "Interaction_PANEL_Resolution",
    "Hands- Free": "Interaction_PANEL_Hands-Free",
    "Eyes- Free": "Interaction_PANEL_Eyes-Free",
    "Possible on One Ear": "Interaction_PANEL_Possible on One Ear",
    "Adaptation of the Interaction Detection Algorithm to the Individual User": "Interaction_PANEL_Adaptation of the Interaction Detection Algorithm to the Individual User",
    "Discreetness of Interaction Techniques": "Interaction_PANEL_Discreetness of Interaction Techniques",
    "Social Acceptability of Interaction Techniques": "Interaction_PANEL_Social Acceptability of Interaction Techniques",
    "Accuracy of Interaction Recognition": "Interaction_PANEL_Accuracy of Interaction Recognition",
    "Robustness of Interaction Detection": "Interaction_PANEL_Robustness of Interaction Detection",
    "Elicitation Study": "Study_PANEL_Elicitation Study",
    "Usability Evaluations": "Study_PANEL_Usability Evaluations",
    "Cognitive Ease Evaluations": "Study_PANEL_Cognitive Ease Evaluations",
    "Discreetness of Interactions Evaluations": "Study_PANEL_Discreetness of Interactions Evaluations",
    "Social Acceptability of Interactions Evaluations": "Study_PANEL_Social Acceptability of Interactions Evaluations",
    "Accuracy of Interactions Evaluation": "Study_PANEL_Accuracy of Interactions Evaluations",
    "Alternative Interaction Validity Evaluations": "Study_PANEL_Alternative Interaction Validity Evaluations",
    "Evaluation of Different Conditions": "Study_PANEL_Evaluation of Different Conditions",
    "Evaluations of Different Settings": "Study_PANEL_Evaluation of Different Settings",
    "Earphone Type": "Device_PANEL_Earphone Type",
    "Development Stage": "Device_PANEL_Development Stage",
    "Real-Time Processing": "Device_PANEL_Real-Time Processing",
    "On Device Processing": "Device_PANEL_On-Device Processing",
    "Intended Applications": "Applications_PANEL_Intended Applications",
    "Keywords": "Keywords",
    "Abstract": "Abstract",
    "Study Link": "Study Link"
}

# Rename the columns
df.rename(columns=new_column_names, inplace=True)

In [9]:
df.to_csv("data.csv", index=False, header=True)